# Spark Apache (семинары)

## Урок 5. Spark on scala

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

- Используя Spark прочитайте данные из файла csv.

- Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

- Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

- Отсортируйте данные по общему объему продаж в порядке убывания.

- Выведите результаты на экран.

In [1]:
import init_spark_env

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as fsum

In [5]:
# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("Books Sales Analysis") \
    .master("local") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 07:48:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [7]:
# Чтение данных из CSV файла
books_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("books.csv")

In [10]:
# Фильтрация книг с продажами более 3000 экземпляров
filtered_books_df = books_df.filter(col("sales") > 3000)

# Группировка по жанру и вычисление общего объема продаж
sales_by_genre_df = filtered_books_df.groupBy("genre") \
    .agg(fsum("sales").alias("total_sales"))

# Сортировка данных по общему объему продаж в порядке убывания
sorted_sales_by_genre_df = sales_by_genre_df.sort(col("total_sales").desc())

# Вывод результатов на экран
sorted_sales_by_genre_df.show()

+------------------+-----------+
|             genre|total_sales|
+------------------+-----------+
| "Science Fiction"|     5000.0|
|           "Novel"|     4500.0|
| "Southern Gothic"|     4000.0|
+------------------+-----------+



In [11]:
spark.stop()